<a href="https://colab.research.google.com/github/claude9493/DSAA5002/blob/main/notebook/Experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/claude9493/dsaa5002.git
!mv ./dsaa5002/dataset/ ./
!mv ./dsaa5002/utils/ ./
!mv ./dsaa5002/models/ ./
!pip install loguru

fatal: destination path 'dsaa5002' already exists and is not an empty directory.
mv: cannot stat './dsaa5002/dataset/': No such file or directory
mv: cannot stat './dsaa5002/utils/': No such file or directory
mv: cannot stat './dsaa5002/models/': No such file or directory


In [2]:
%%time
import sys
import datetime
import time
from utils.utils import create_dataset, Trainer
from models.model import *
# from layer.layer import Embedding, FeaturesEmbedding, EmbeddingsInteraction, MultiLayerPerceptron

import math
import numpy as np
from loguru import logger
from tqdm.autonotebook import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ConstantLR

from torch.utils.data import TensorDataset, DataLoader

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Training on [{}].'.format(device))

Training on [cuda:0].
CPU times: user 866 ms, sys: 390 ms, total: 1.26 s
Wall time: 1.14 s


In [11]:
import importlib
from models import model
from models.model import *
importlib.reload(model)
from models.model import *

In [3]:
%%time

# Load data and split train, validation, test sets.
SAMPLE_NUM = 10000
task = 'regression'  # 'classification'
dataset = create_dataset('movielens', sample_num=SAMPLE_NUM, task=task, device=device)
field_dims, (train_X, train_y), (valid_X, valid_y), (test_X, test_y) = dataset.train_valid_test_split()

CPU times: user 1.6 s, sys: 808 ms, total: 2.4 s
Wall time: 2.4 s


In [4]:
config = dict(
    EMBEDDING_DIM = 50,
    LEARNING_RATE = 0.25, #1e-4
    REGULARIZATION = 1e-6,
    BATCH_SIZE = 1024,  # 64
    EPOCH = 1200,
    TRIAL = 200)

In [5]:
def train(model_name, config, 
          scheduler=None, draw=False,
          train_X=train_X, train_y=train_y, 
          valid_X=valid_X, valid_y=valid_y, 
          test_X=test_X, test_y=test_y, **kwargs):

    EMBEDDING_DIM = config.get('EMBEDDING_DIM')
    LEARNING_RATE = config.get('LEARNING_RATE')
    REGULARIZATION = config.get('REGULARIZATION')
    BATCH_SIZE = config.get('BATCH_SIZE')
    EPOCH = config.get('EPOCH')
    TRIAL = config.get('TRIAL')


    models = dict(
        MF = MatrixFactorization,
        EE = EuclideanEmbedding,
        BR = BetaRecommendation
    )

    model = models.get(model_name)(field_dims, EMBEDDING_DIM, **kwargs).to(device)

    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=REGULARIZATION)

    if scheduler == 'const':
        scheduler = ConstantLR(optimizer, factor=0.2, total_iters=300)
    
    criterion = dict(
        MF = RMSELoss(),
        EE = EE_Loss(model=model),
        BR = Beta_Loss(model=model)
    )[model_name]
        
    trainer = Trainer(model, optimizer, criterion, BATCH_SIZE, scheduler=scheduler, task=task, draw=draw)
    train_loss, valid_loss, iters = trainer.train(train_X, train_y, 
                                           valid_X=valid_X, valid_y=valid_y, 
                                           epoch=EPOCH, trials=TRIAL, log=True)
    test_loss, test_metric = trainer.test(test_X, test_y)
    # print('test_loss:  {:.5f} | test_metric:  {:.5f}'.format(test_loss, test_metric))
    logger.success('test_loss:  {:.5f} | test_metric:  {:.5f}'.format(test_loss, test_metric))
    return (train_loss.detach().item(), valid_loss.detach().item(), test_loss.detach().item()), iters


def train_repeat(model, config, 
                 scheduler=None, R=10, 
                 dataset=dataset, **kwargs):
  losses = []
  pbar = tqdm(range(R))
  for _ in pbar:
    field_dims, (train_X, train_y), (valid_X, valid_y), (test_X, test_y) = dataset.train_valid_test_split()
    t0 = time.time()
    loss, iters = train(model, config, scheduler, draw=False,
                        train_X=train_X, train_y=train_y, 
                        valid_X=valid_X, valid_y=valid_y, 
                        test_X=test_X, test_y=test_y, **kwargs)
    t1 = time.time()
    total_time = t1-t0
    losses.append((*loss, total_time, iters))
    pbar.set_description("Train loss: {:.3f} | Validation loss {:.3f} | Test loss {:.3f} | {:.3f}s - {}".format(*losses[-1]))
  losses = np.array(losses)
  # print(losses)
  return losses.mean(axis=0), losses.std(axis=0)

In [6]:
%%time
for ed in [50, 25, 5]:
  logger.remove(handler_id=None)
  logger.add(sink=f"MF_{ed}_{time.time()}.log")
  loss_mean, loss_std = train_repeat('MF', {**config, 'EMBEDDING_DIM':ed, 'LEARNING_RATE':0.005}, R=50)
  print("Matrix Factorization ({})\nAverage RMSE:{}\tStandard deviation:{}".format(ed, loss_mean, loss_std))
  print(np.array([loss_mean, loss_std]))

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/1200 [00:00<?, ?it/s]

  0%|          | 0/1200 [00:00<?, ?it/s]

  0%|          | 0/1200 [00:00<?, ?it/s]

  0%|          | 0/1200 [00:00<?, ?it/s]

...

  0%|          | 0/1200 [00:00<?, ?it/s]

  0%|          | 0/1200 [00:00<?, ?it/s]

  0%|          | 0/1200 [00:00<?, ?it/s]

  0%|          | 0/1200 [00:00<?, ?it/s]

Matrix Factorization (5)
Average RMSE:[  0.33779662   1.67848635   1.74481661   4.32305273 233.02      ]	Standard deviation:[0.00688116 0.00079277 0.00085373 0.06301901 0.64776539]
[[3.37796623e-01 1.67848635e+00 1.74481661e+00 4.32305273e+00
  2.33020000e+02]
 [6.88116411e-03 7.92773308e-04 8.53729785e-04 6.30190142e-02
  6.47765390e-01]]
CPU times: user 18min 47s, sys: 39.9 s, total: 19min 27s
Wall time: 20min 47s


In [9]:
%%time
for ed in [50, 25, 5]:
  logger.remove(handler_id=None)
  logger.add(sink=f"EE_{ed}_{time.time()}.log")
  loss_mean, loss_std = train_repeat('EE', {**config, 'EMBEDDING_DIM':ed, 'LEARNING_RATE':0.005}, R=50)
  print("Euclidean Embedding ({})\nAverage RMSE:{}\tStandard deviation:{}".format(ed, loss_mean, loss_std))
  print(np.array([loss_mean, loss_std]))

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/1200 [00:00<?, ?it/s]

  0%|          | 0/1200 [00:00<?, ?it/s]

  0%|          | 0/1200 [00:00<?, ?it/s]

  0%|          | 0/1200 [00:00<?, ?it/s]

...

  0%|          | 0/1200 [00:00<?, ?it/s]

  0%|          | 0/1200 [00:00<?, ?it/s]

  0%|          | 0/1200 [00:00<?, ?it/s]

  0%|          | 0/1200 [00:00<?, ?it/s]

Euclidean Embedding (5)
Average RMSE:[  1.02994284   1.30264071   1.32400378  13.74627351 404.2       ]	Standard deviation:[6.05316580e-04 9.10959880e-03 9.70421546e-03 5.20929397e-01
 1.45876660e+01]
[[1.02994284e+00 1.30264071e+00 1.32400378e+00 1.37462735e+01
  4.04200000e+02]
 [6.05316580e-04 9.10959880e-03 9.70421546e-03 5.20929397e-01
  1.45876660e+01]]
CPU times: user 47min 14s, sys: 1min 14s, total: 48min 29s
Wall time: 51min 35s


In [ ]:
%%time
for ed in [50, 25, 5]:
  logger.remove(handler_id=None)
  logger.add(sink=f"BR_{ed}_{time.time()}.log")
  loss_mean, loss_std = train_repeat('BR', {**config, 'EMBEDDING_DIM':ed}, 
                                     R=50, scheduler='const', dist_metric='JS')
  print("Beta Recommendation ({})\nAverage RMSE:{}\tStandard deviation:{}".format(ed, loss_mean, loss_std))
  print(np.array([loss_mean, loss_std]))

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/1200 [00:00<?, ?it/s]

  0%|          | 0/1200 [00:00<?, ?it/s]

  0%|          | 0/1200 [00:00<?, ?it/s]